In [ ]:
%load_ext autoreload 
%autoreload 2
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.integrate import cumulative_trapezoid, solve_ivp, solve_bvp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import cmocean


from utility import internal_bore, bore_model, J_boreheight, J_borespeed, J_boremomn

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{lmodern}'

### Define bore parameters

In [ ]:
H = 50
h0 = 100
cb = 0.52
ib = internal_bore(H = H, H_0 = h0, z0 = 0.7, ε = 0) #, d0 = 1e-5,  ε = -1e-2, rho_s = 1024, rho_b = 1024.1, rho_0 = 1024.05)

### Compute solution: IVP Method

In [4]:
def bc(ya, yb,p):
    return np.array([ya[0], yb[0], p[0]])

y0 = np.zeros((2,ib.z.size))
M = solve_bvp(ib.ode_bvp, bc, ib.z, y0, p = [cb], max_nodes=1e4, verbose = 0)
M

NameError: name 'np' is not defined

### Compute Solution IVP Method

In [3]:
#Compute solution: IVP method

dn0 = 1e-1
M = solve_ivp(ib.ode_ivp, [0,ib.H], [0, dn0], args = (cb,), dense_output = True, method = "Radau")
ib.n, ib.nz = M.sol(ib.z)[0], M.sol(ib.z)[1]
ib.z = ib.za + ib.n #displaced isopycnals
ib.b, ib.bz = ib.buoyancy(ib.z + ib.H_0 - ib.H)

display( np.trapz(ib.b, ib.z) )
#display(ib.nz)
#n = ib.streamline_analytic(N = 1e-3, c = .5)

#M

NameError: name 'solve_ivp' is not defined

## Cross-shore structure of energy flux divergence

Also of interest is the cross-shore structure of the energy flux divergence itself. We plot $F_x(h)$, but now against the same  coordinate ($x = 0$ at the 100m isobath and increases shoreward) as in Becherer et al. 2021(a) Figure 7 to invite comparison. A constant slope of $0.01$, characteristic of the region is assumed.

<!-- ![diss](B21_fig7.jpg) -->
<img src="B21_fig7.jpg" alt="Drawing" style="width: 750px;"/>

Subpanel b) shows the energy flux divergence and vertically integrated dissipation rate plotted against across-shore distance

In [ ]:
%matplotlib widget
from matplotlib.colors import TwoSlopeNorm

m = 0.01 #bottom slope
x0 = 100/m #offshore distance of 100m isobath given slope (m) $h = -m*(x-x0) + 100

fig, axes = plt.subplots(2,2,figsize = (6.5,6))

h_s = 90 #shallow isobath
h_d = 100 #deep isobath

#plot Becherer 2021a best fit line
hh = np.linspace(h_s, h_d,1000)
xx = -(hh - 100)/m 
xx *= 1e-3
yy = np.exp(-0.013*xx**2 - 0.18*xx + 5.0)

plt.subplot(2, 1, 1)
plt.plot(xx,yy/1e3, color = "k", label = "B21 polynomial fit")

nd = 4
h = np.linspace(h_s, h_d, nd) #depths
rho = np.zeros((100,nd))
X = np.zeros((100,nd))
Z = np.zeros((100,nd))
N2 = np.zeros((100,nd))

#compute and plot W&H14 solution
for k, f in enumerate(h):
    display(f)

    ib = internal_bore(H = f)
    F_hb = minimize_scalar(bore_model, args = (ib,), method = "bounded", bounds = bounds, options = dict(maxiter = 50, xatol=1e-2)) #maximize energy loss
    IB = bore_model(h_b = f/2, ib = ib, output = "ib", disp = False)
    
    x = -(f - 100)/m
    ps = plt.scatter(x/1e3, IB.D, color = "k", facecolors = "none")

    rho[:,k] = IB.density()
    N2[:,k] = IB.bvf()
    X[:,k] = x
    Z[:,k] = IB.zb - IB.zb.max()
    
ps.set_label("W\&H14")
plt.gca().set_xlabel("x (km)", fontsize = 14)
plt.gca().set_ylabel("$F_x$", rotation = 0, fontsize = 14, labelpad = 15)
plt.gca().set_yscale("log")
plt.gca().legend(frameon = False)

#plot density
pc = axes[1,0].pcolormesh(X/1e3, Z, rho - 1000, cmap = "cmo.dense", shading = "nearest")
axes[1,0].contour(X/1e3, Z, rho - 1000, 20, colors = "w", linewidths = 0.5)
plt.colorbar(pc, ax = axes[1,0], orientation = "horizontal", pad = 0.25)

#compute geostrophic velocity (thermal wind)
f0 = 1e-4
v = cumulative_trapezoid( -(N2/f0)*np.gradient(Z, axis = 1)/np.gradient(X, axis = 1), Z, axis = 0, initial = 0)

#plot geostrophic velocity
pc = axes[1,1].pcolormesh(X/1e3, Z, v, cmap = "cmo.balance", shading = "nearest", norm = TwoSlopeNorm(vcenter = 0))
plt.colorbar(pc, ax = axes[1,1], orientation = "horizontal", pad = 0.25)

for ax in axes.flatten():
    #ax.set_xlim(0,13.5)
    ax.set_xlabel("x (km)", fontsize = 14)

# axes[1,0].set_xlim(0, 13.5)
axes[1,0].set_ylabel("z (m)", rotation = 0, fontsize = 14, labelpad = 20)
axes[1,0].set_facecolor("gainsboro")

axes[1,1].set_facecolor("gainsboro")
axes[1,1].yaxis.set_visible(False)

axes[0,1].set_visible(False)
fig.tight_layout()
sns.despine()

## Bore Amplitude.

In W&H14, theoretical predictions were compared to numerical model results, so the desired bore amplitude for use in the iterative procedure was prescribed. Here, however, we must come up with a bore amplitude estimate _a priori_. By applying the W&H14 bore model, we find an $\widetilde{H}_b$ for a given depth and buoyancy profile that maximizes the depth-integrated energy loss ($F_x$), and also be physically realistic ($F_x > 0$). This approach is consistent with the W&H14 finding that the best analytical model fit to simulation involved using a head loss function that maximized dissipation ($\epsilon \rightarrow -\infty$).

** do we actually need to maximize dissipation or just aassume h_b = 0.5*H?

$$ b(z) = \frac{\rho(z) - \rho(H)}{\rho(0) - \rho(H)} $$

$$ b(z) = \frac{1}{2} - \frac{1}{2}\tanh{[\lambda(z-z_0)]}$$

$$ b'(z) = \frac{\lambda }{2}\text{sech}^2{[\lambda(z-z_0)]}$$

$$\rho_a(z) = \frac{1}{2} - \frac{1}{2}\tanh{[\lambda(z-h_a)]}$$

$$z_0(x\rightarrow 0) = \frac{1}{2}(h_a + h_d)$$

$$h_b = \int^H_0 b(z)\,dz$$

$$\frac{d}{dz} = (1-\eta_z)\frac{d}{dz_a}$$

$$z_a = z - \eta$$

$$ <D> = \int^T_0 \frac{D_0}{2}\big[\sin(\frac{2\pi t}{T}) + 1\big] dt $$  
$$ <D> = \frac{D_0}{T}\int^T_0 \text{ sech}(\sin\frac{\pi t}{T})dt $$  

thermal wind as related to BVF
$$ f \frac{dv}{dz} = N^2\frac{d z_{rho}}{dx} $$

In [7]:
def periodic_func(t,T, λ = 10, phase = 0):
    return 1/np.cosh(λ*np.sin( (np.pi*(t-phase))/T))

In [ ]:
%matplotlib widget
T = 12.42*3600/2
λ = 10
t = np.linspace(0,T*4,1000)
f = periodic_func(t,T, λ = λ, phase = T/2 )

plt.figure(figsize = (5,1.5))
plt.plot(t/3600,f,"k")
cff = np.trapz(f, t)/T